<a href="https://colab.research.google.com/github/btlgs2000/dl_intro/blob/master/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from tensorflow import keras

In [1]:
! git clone https://github.com/btlgs2000/dl_intro

Cloning into 'dl_intro'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 41 (delta 17), reused 31 (delta 11), pack-reused 0
Unpacking objects: 100% (41/41), done.


In [2]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vL3uIpBcd-U_xnKD4EKvXGyL7r66CuDJ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1vL3uIpBcd-U_xnKD4EKvXGyL7r66CuDJ" -O cats_dogs.zip && rm -rf /tmp/cookies.txt

--2020-08-28 12:04:19--  https://docs.google.com/uc?export=download&confirm=4L78&id=1vL3uIpBcd-U_xnKD4EKvXGyL7r66CuDJ
Resolving docs.google.com (docs.google.com)... 108.177.126.138, 108.177.126.100, 108.177.126.101, ...
Connecting to docs.google.com (docs.google.com)|108.177.126.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-8c-docs.googleusercontent.com/docs/securesc/fhfpif17rbb60tu6889biscf1s55icu2/vp678sv7rgufps57h6a1fm4c312cv6ev/1598616225000/10454975649907589971/10631540946199132849Z/1vL3uIpBcd-U_xnKD4EKvXGyL7r66CuDJ?e=download [following]
--2020-08-28 12:04:19--  https://doc-0o-8c-docs.googleusercontent.com/docs/securesc/fhfpif17rbb60tu6889biscf1s55icu2/vp678sv7rgufps57h6a1fm4c312cv6ev/1598616225000/10454975649907589971/10631540946199132849Z/1vL3uIpBcd-U_xnKD4EKvXGyL7r66CuDJ?e=download
Resolving doc-0o-8c-docs.googleusercontent.com (doc-0o-8c-docs.googleusercontent.com)... 108.177.119.132, 2a00:1450:4013:c00::84
Conne

In [3]:
! unzip /content/cats_dogs.zip

Streaming output truncated to the last 5000 lines.
  inflating: dogs-vs-cats/train/dog/dog.55.jpg  
  inflating: dogs-vs-cats/train/dog/dog.550.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5500.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5501.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5502.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5503.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5504.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5505.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5506.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5507.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5508.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5509.jpg  
  inflating: dogs-vs-cats/train/dog/dog.551.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5510.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5511.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5512.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5513.jpg  
  inflating: dogs-vs-cats/train/dog/dog.5514.jpg  
  inflating: dogs-vs-cats/train/dog

In [7]:
TRAIN_FOLDER = r'/content/dogs-vs-cats/train'
BATCH_SIZE = 32
IMAGE_SIZE = (64, 64)

In [8]:
train_ds = keras.preprocessing.image_dataset_from_directory(
    directory=TRAIN_FOLDER,
    labels='inferred',
    label_mode='binary',
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    seed=1,
    validation_split=0.1,
    subset='training')

val_ds = keras.preprocessing.image_dataset_from_directory(
    directory=TRAIN_FOLDER,
    labels='inferred',
    label_mode='binary',
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    seed=1,
    validation_split=0.1,
    subset='validation')

Found 25000 files belonging to 2 classes.
Using 22500 files for training.
Found 25000 files belonging to 2 classes.
Using 2500 files for validation.
